In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<h1>
    Import raw data files
</h1>

In [2]:
literacy_raw = pd.read_excel("./up_raw_literacy_cleaned_9_Sep_4_Oct_v4.xlsx", index_col='_id')
numeracy_raw = pd.read_excel("./up_raw_numeracy_cleaned_9_Sep_4_Oct_v4.xlsx", index_col='_id')

In [3]:
def total_score(scores):
    total_score = 0
    for score in scores:
        if score == 1:
            total_score += 1
    return total_score

In [4]:
general_info =['tabletUserName', 'assessment_date', 'school_details.State_label', 'school_details.District_label', \
               'school_details.Block_label', 'school_details.School_label', 'school_details.UDISE_cd_label']

student_info = ['SI_std_name', 'student_age', 'student_gender']

up_lit_scores = literacy_raw[general_info + student_info].set_index(literacy_raw.index)
up_num_scores = numeracy_raw[general_info + student_info].set_index(numeracy_raw.index)

<h1 style="color:blue;">
    Literary Sub-task Score Calculations
</h1>

<h3>
    Literacy 1: Listening Comprehension
</h3>

In [5]:
# Calculate total score on listening comprehension sub-task
literacy1 = [col for col in literacy_raw.columns if re.search(r'literacy1_q\d$', col)]
                                                         
up_lit_scores.loc[:, 'literacy1_total'] = literacy_raw.apply(lambda x: total_score([x[col] for col in literacy1]), axis=1)

In [6]:
# Extract other responses to oral vocabulary questions
literacy1_or = [col for col in literacy_raw.columns if re.search(r'literacy1\S*or$', col)]
with pd.ExcelWriter('literacy1_other_responses.xlsx') as writer: 
    for col in literacy1_or:
        literacy_raw[col].value_counts().reset_index().rename(columns = {'index':"Child's response", col:'Frequency'}).to_excel(writer, sheet_name=col)

<h3>
    Literacy 2: Oral Vocabulary
</h3>

In [7]:
# Calculate total score on oral comprehension sub-task
literacy2 = [col for col in literacy_raw.columns if re.search(r'literacy2_q\d+$', col)]
                                                         
up_lit_scores.loc[:, 'literacy2_total'] = literacy_raw.apply(lambda x: total_score([x[col] for col in literacy2]), axis=1)

In [8]:
# Extract other responses to oral vocabulary questions
literacy2_or = [col for col in literacy_raw.columns if re.search(r'literacy2\S*or$', col)]
with pd.ExcelWriter('literacy2_other_responses.xlsx') as writer: 
    for col in literacy2_or:
        literacy_raw[col].value_counts().reset_index().rename(columns = {'index':"Child's response", col:'Frequency'}).to_excel(writer, sheet_name=col)

<h3>
    Literacy 3: Initial Sound Identification
</h3>

In [9]:
# Calculate total score on initial sound identification sub-task
literacy3 = [col for col in literacy_raw.columns if re.search(r'literacy3_q+', col)]
                                                         
up_lit_scores.loc[:, 'literacy3_total'] = literacy_raw.apply(lambda x: total_score([x[col] for col in literacy3]), axis=1)

<h3>
    Literacy 4: Letter Recognition (Untimed)
</h3>

In [10]:
literacy4_ut = [col for col in literacy_raw.columns if re.search(r'literacy4_ut_grid_\d*$', col)]
    
# Calculate total score on letter naming (untimed) sub-task
up_lit_scores.loc[:, 'literacy4_ut_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy4_ut]), axis=1)

<h3>
    Literacy 4: Letter Recognition (Timed)
</h3>

In [11]:
literacy4_tt =  [col for col in literacy_raw.columns if re.search(r'literacy4_tt_grid_\d*$', col)]

# Calculate total score on letter naming (timed) sub-task
up_lit_scores.loc[:, 'literacy4_tt_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy4_tt]), axis=1)
# mp_lit_scores.loc[:, 'literacy4_tt_time_remaining'] = literacy_raw['literacy4_tt_grid.time_remaining']

<h3>
    Literacy 5: Familiar Words Reading (Untimed)
</h3>

In [12]:
literacy5_ut = [col for col in literacy_raw.columns if re.search(r'literacy5_ut_grid_\d*$', col)]
    
up_lit_scores.loc[:, 'literacy5_ut_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy5_ut]), axis=1)

<h3>
    Literacy 5: Familiar Words Reading (Timed)
</h3>

In [13]:
literacy5_tt = [col for col in literacy_raw.columns if re.search(r'literacy5_tt_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy5_tt_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy5_tt]), axis=1)

<h3>
    Literacy 6: Non-word Reading
</h3>

In [14]:
literacy6 = [col for col in literacy_raw.columns if re.search(r'literacy6_tt_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy6_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy6]), axis=1)

<h3>
    Literacy 7: Oral Reading Fluency (Timed)
</h3>

In [15]:
literacy7 = [col for col in literacy_raw.columns if re.search(r'literacy7_tt_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy7_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy7]), axis=1)

<h3>
    Literacy 8: Reading Comprehension (Untimed)
</h3>

In [16]:
literacy8_reading = [col for col in literacy_raw.columns if re.search(r'literacy8_ut_grid_\d*$', col)]

literacy8_comprehension = [col for col in literacy_raw.columns if re.search(r'literacy8_ut_q\d*$', col)]

up_lit_scores.loc[:, 'literacy8_reading_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy8_reading]), axis=1)

up_lit_scores.loc[:, 'literacy8_comprehension_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy8_comprehension]), axis=1)

<h3>
    Literacy 9a: Dictation (Letters)
</h3>

In [17]:
literacy9a = [col for col in literacy_raw.columns if re.search(r'literacy9a_ut_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy9a_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy9a]), axis=1)

<h3>
    Literacy 9a: Dictation (Words)
</h3>

In [18]:
literacy9b = [col for col in literacy_raw.columns if re.search(r'literacy9b_ut_grid_\d*$', col)]

up_lit_scores.loc[:, 'literacy9b_total'] = literacy_raw.apply(lambda x: total_score([x[score] for score in literacy9b]), axis=1)

In [19]:
up_lit_scores

,tabletUserName,assessment_date,school_details.State_label,school_details.District_label,school_details.Block_label,school_details.School_label,school_details.UDISE_cd_label,SI_std_name,student_age,student_gender,literacy1_total,literacy2_total,literacy3_total,literacy4_ut_total,literacy4_tt_total,literacy5_ut_total,literacy5_tt_total,literacy6_total,literacy7_total,literacy8_reading_total,literacy8_comprehension_total,literacy9a_total,literacy9b_total
_id,,,,,,,,,,,,,,,,,,,,,,,
005680d6-db55-461c-9008-ecf1bc215e6a,Durgawati,21-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS VIHARA,9670407301,Priyanshu,6,Male,0,9,2,13,37,1,14,2,0,0,0,8,0
006223ab-87fe-499b-bc7d-b85b1edb102a,Snow,20-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS NEWADA BABHNAUT,9670402001,Arpita patel,7,Female,2,10,3,13,50,2,24,13,0,0,0,10,0
009328d2-607b-44b0-8464-65264e77c3cc,Kusum,22-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS SATTANPUR,9670408201,Sumit,6,Male,3,9,10,13,44,5,16,12,3,1,0,8,6
00b6a9dd-71d5-4c03-9e1d-9360898fbf68,Rohanyadav,22-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS RAMESEWAR (I),9670405701,Divyansh yadav,5,Male,1,8,0,6,23,0,0,0,0,0,0,6,0
00fce497-e572-4b84-8053-b35227dbb24b,Geetakumari,23-09-2022,Uttar Pradesh,Varanasi,Kashividyapeeth,PS SARAI DANGARI,9670102201,shiwansh,6,Male,2,7,0,4,9,0,0,0,0,0,0,2,0
01f7b9cb-a1a1-4874-acd0-ef2ffc21db50,Sanyogita,23-09-2022,Uttar Pradesh,Varanasi,Kashividyapeeth,"PS NARAUR, BARASUR",9670102501,Abhay,6,Male,3,9,0,12,32,4,16,6,2,33,0,6,0
02a40947-fd38-4723-bafa-6b90fc173126,Rohanyadav,20-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS RAMPUR (II) (PURE BARIAR),9670403201,varis,6,Male,2,8,1,0,2,0,0,0,0,0,0,0,0
02b669c5-b14a-4800-ac3b-1a3a49948431,Sarika,21-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS RAGHUNATH PUR,9670400701,Chanda,6,Female,0,5,0,0,0,0,0,0,0,0,0,0,0
033e7e64-5fdd-4e30-bb1b-39df5f84bdea,Vinita,22-09-2022,Uttar Pradesh,Varanasi,Sewapuri,PS GADDOPUR,9670414701,Aysa saroj,5,Female,3,10,0,0,2,0,0,0,0,0,0,0,0


In [20]:
up_lit_scores.to_excel("up_literacy_total_scores_v2.xlsx")

<h2>
    B. Numeracy Sub-tasks Data Cleaning
</h2>

<h3>
    Numeracy 1: Counting
</h3>

In [21]:
# Calculate total score on counting sub-task
numeracy1 = [col for col in numeracy_raw.columns if re.search(r'numeracy1_tt_grid_\d*$', col)]

up_num_scores.loc[:, 'numeracy1_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy1]), axis=1)

<h3>
    Numeracy 2: Number Recognition (Untimed)
</h3>

In [22]:
numeracy2_ut = [col for col in numeracy_raw.columns if re.search(r'numeracy2_ut_grid_\d*$', col)]

up_num_scores.loc[:, 'numeracy2_ut_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy2_ut]), axis=1)

<h3>
    Numeracy 2: Number Recognition (Timed)
</h3>

In [23]:
numeracy2_tt = [col for col in numeracy_raw.columns if re.search(r'numeracy2_tt_grid_\d*$', col)]

up_num_scores.loc[:, 'numeracy2_tt_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy2_tt]), axis=1)

<h3>
    Numeracy 3: Number Comparison
</h3>

In [24]:
numeracy3 = [col for col in numeracy_raw.columns if re.search(r'numeracy3\w*', col)]

up_num_scores.loc[:, 'numeracy3_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy3]), axis=1)

<h3>
    Numeracy 4: Counting in Bundles
</h3>

In [25]:
numeracy4 = [col for col in numeracy_raw.columns if re.search(r'numeracy4_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy4_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy4]), axis=1)

<h3>
    Numeracy 5: Missing Numbers
</h3>

In [26]:
numeracy5 = [col for col in numeracy_raw.columns if re.search(r'numeracy5_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy5_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy5]), axis=1)

<h3>
    Numeracy 6: Addition
</h3>

In [27]:
numeracy6 = [col for col in numeracy_raw.columns if re.search(r'numeracy6_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy6_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy6]), axis=1)

<h3>
    Numeracy 7: Subtraction
</h3>

In [28]:
numeracy7 = [col for col in numeracy_raw.columns if re.search(r'numeracy7_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy7_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy7]), axis=1)

<h3>
    Numeracy 8: Word Problems
</h3>

In [29]:
numeracy8 = [col for col in numeracy_raw.columns if re.search(r'numeracy8_ut_q\d*$', col)]

up_num_scores.loc[:, 'numeracy8_total'] = numeracy_raw.apply(lambda x: total_score([x[col] for col in numeracy8]), axis=1)

<h3>
    Numeracy 9a: Shape Recognition (Circle)
</h3>

In [30]:
numeracy9a = [col for col in numeracy_raw.columns if re.search(r'numeracy9a_ut_grid_\d$', col)]

def total_score_9a(scores):
    total_score_9a = 0
    if scores[0] == '1' and scores[1] == '1' and scores[2] == '0' and scores[3] == '1' and scores[4] == '1' and scores[5] == '1' and scores[6] == '1' and scores[7] == '0':
            total_score_9a += 1 
    return total_score_9a
    
up_num_scores.loc[:, 'numeracy9a_total'] = numeracy_raw.apply(lambda x: total_score_9a([x[score] for score in numeracy9a]), axis=1)

<h3>
    Numeracy 9b: Shape Recognition (Rectangle)
</h3>

In [31]:
numeracy9b = [col for col in numeracy_raw.columns if re.search(r'numeracy9b_ut_grid_\d$', col)]
    
def total_score_9b(scores):
    total_score_9b = 0
    if scores[0] == '1' and scores[1] == '0' and scores[2] == '1' and scores[3] == '1' and scores[4] == '0' and scores[5] == '1' and scores[6] == '1' and scores[7] == '0':
            total_score_9b += 1 
    return total_score_9b   
    
up_num_scores.loc[:, 'numeracy9b_total'] = numeracy_raw.apply(lambda x: total_score_9b([x[score] for score in numeracy9b]), axis=1)

In [32]:
up_num_scores.to_excel("mp_numeracy_total_scores_v2.xlsx")